In [1]:
import functies as fn
import pandas as pd
from IPython.display import Image, display
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import numpy as np
from sklearn.metrics import r2_score, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.linear_model as lm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import graphviz
from sklearn.mixture import GaussianMixture as gmm
from sklearn.cluster import KMeans

In [2]:
dfr = pd.read_csv('data/movie-1.csv')

# Toon de maximale informatie die de dataframe kan geven.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Formatteer alle grote getallen voor een betere leesbaarheid.
pd.set_option('display.float_format', '{:,.2f}'.format)

## Onderzoeksvraag 3: Hoe kunnen budget en omzet worden gebruikt om logische clusters van de films te vinden?


In [3]:
df = dfr.copy()

Om clusters te bepalen in een dataset gaan we nu gebruik maken van unsupervised learning. De algoritmes die we gaan toepassen zijn KMeans en GMM.

We maken een nieuw dataframe aan met 'gross' en 'budget'.

In [4]:
df_gross_budget = df[['gross', 'budget']]

Bekijken hoeveel NaN values in de dataset voorkomen.

In [ ]:
df_gross_budget.isna().sum()


Bekijken hoeveel rows het volledige df bevat. Als de NaN waardes maar een klein percentage van de volledige dataset bevat kunnen deze worden verwijderd. Als het een groot percentage bevat dan moeten we kijken of we dit logisch in kunnen vullen

In [ ]:
total_rows = len(df_gross_budget)
print(f'aantal rows in de totale datasframe: {total_rows}')
nan_rows = df_gross_budget.isna().any(axis=1).sum()
print(f'aantal nan rows in de totale datasframe: {nan_rows}')
nan_percentage_rows = ((nan_rows / total_rows) * 100).round(1)
print(f'Bevat NaN rows t.o.v. totale dataframe: {nan_percentage_rows}%')

# Plotten
labels = ['Rijen met NaN', 'Rijen zonder NaN']
values = [nan_rows, total_rows - nan_rows]

plt.figure(figsize=(8, 6))
plt.pie(values, labels=labels, autopct='%1.1f%%', startangle=90, colors=['lightcoral', 'lightgreen'], explode=[0.1, 0])
plt.title('Verdeling van rijen met en zonder NaN-waarden', fontsize=14)
plt.tight_layout()
plt.show()

Dit is een aanzienlijk groot percentage NaN's. We gaan de data opvullen. Maar eerst moeten we bekijken welke kolommen uit de dataset het meest correleren. De kolommen die het meest correleren met gross en budget kunnen worden gebruikt om de missende data in te vullen.

In [ ]:
# Selecteer numerieke kolommen voor correlatie analyse
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
correlation_matrix = df[numeric_columns].corr()

# Plot correlatie heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, 
            annot=True,
            cmap='coolwarm',
            center=0,
            fmt='.2f')
plt.title('Correlatie tussen numerieke variabelen')
plt.tight_layout()
plt.show()

# Print de top 5 correlaties met budget en gross
print("\nTop 5 correlaties met budget:")
budget_correlations = correlation_matrix['budget'].sort_values(ascending=False)
print(budget_correlations.head())

print("\nTop 5 correlaties met gross:")
gross_correlations = correlation_matrix['gross'].sort_values(ascending=False)
print(gross_correlations.head())

De enige matige correlaties zijn tussen de gross en num_voted_users en num_users_for_reviews. We zullen deze 2 kolommen gebruiken om de missende NaN values in te vullen van gross. 

In [8]:
# Vul missende waarden van 'gross' in op basis van gemiddelden per cluster
# Cluster bepalen op basis van 'num_voted_users' en 'num_user_for_reviews'

# Maak een nieuwe kolom voor clustering (bijvoorbeeld afronden van aantallen gebruikers)
df['cluster'] = (
    df['num_voted_users'].fillna(0).round(-3).astype(int).astype(str) + '_' +
    df['num_user_for_reviews'].fillna(0).round(-2).astype(int).astype(str)
)

# Bereken gemiddelde 'gross' per cluster
cluster_gross_mean = df.groupby('cluster')['gross'].mean()

# Vul NaN-waarden van 'gross' in met de gemiddelden per cluster
df['gross'] = df.apply(
    lambda row: cluster_gross_mean[row['cluster']] if pd.isna(row['gross']) else row['gross'],
    axis=1
)

# Verwijder tijdelijke clusterkolom
df.drop(columns=['cluster'], inplace=True)


Weer opnieuw een dataframe maken met alleen gross en budget maar dan met de ingevulde gross kolom

In [9]:
df = df[['gross', 'budget']]

Weer alle NaN values checken

In [ ]:
df.isna().sum()

Opnieuw bekijken hoeveel procent NaN waarden bevat

In [ ]:
total_rows = len(df)
print(f'aantal rows in de totale datasframe: {total_rows}')
nan_rows = df.isna().any(axis=1).sum()
print(f'aantal nan rows in de totale datasframe: {nan_rows}')
nan_percentage_rows = ((nan_rows / total_rows) * 100).round(1)
print(f'Bevat NaN rows t.o.v. totale dataframe: {nan_percentage_rows}%')

# Plotten
labels = ['Rijen met NaN', 'Rijen zonder NaN']
values = [nan_rows, total_rows - nan_rows]

plt.figure(figsize=(8, 6))
plt.pie(values, labels=labels, autopct='%1.1f%%', startangle=90, colors=['lightcoral', 'lightgreen'], explode=[0.1, 0])
plt.title('Verdeling van rijen met en zonder NaN-waarden', fontsize=14)
plt.tight_layout()
plt.show()

We hebben het aantal NaN values verlaagd van 22.8% tot 10.1%. Dit is nogsteeds relatief veel maar helaas is er weinig correlatie met de budget kolom waardoor we deze niet verder in kunnen vullen. Daarom kiezen we ervoor om de resterende NaN waarden te verwijderen.

Alle NaN's verwijderen

In [12]:
df = df.dropna()

Opnieuw bekijken hoeveel NaN's in de dataset voorkomen. Na het verwijderen ervan moeten ze voor allebei op 0 staan.

In [ ]:
df.isna().sum()

Het dataframe bekijken

In [ ]:
df.head(10)

## Verwijderen van outliers ##

Omdat de modellen gevoelig zijn voor outliers gaan we identificeren welke we gaan verwijderen.

In [ ]:
fn.boxplot_gross_and_budget(df, 'gross')
fn.histogram_gross(df, 'gross')

fn.boxplot_gross_and_budget(df, 'budget')
fn.histogram_gross(df, 'budget')

Als we kijken naar een combinatie van de boxplots en histogrammen, dan zien we dat er extreme outliers in onze dataset voorkomen. Onze onderzoeksvraag vraagt niet om bijzondere gevallen te bepalen, maar om logische clusters te bepalen, wat na verwijderen van de outliers nogsteeds kan worden gedaan. Hierdoor besluiten we alle outliers te verwijderen uit onze dataset. Hierbij maken we gebruik van de functies uit functies.py. Ook bekijken we het aantal rijen en kolommen voordat we ze gaan verwijderen om te checken of ze echt zijn verwijderd.

In [ ]:
df.shape

In [17]:
df = fn.remove_outliers(df, 'gross')
df = fn.remove_outliers(df, 'budget')

Checken of de outliers zijn verwijderd door het aantal rijen en kolommen te bekijken. Dit is inderdaad een lager getal dan eerst, wat betekend dat ze zijn verwijderd.

In [ ]:
df.shape

Ook bekijken we de histogrammen opnieuw. De x as moet een bereik hebben wat overeenkomt met wat we op de boxplots zagen. 

In [ ]:
fn.histogram_gross(df, 'gross')

fn.histogram_gross(df, 'budget')

Het bereik op de x as komt inderdaad overeen met wat we zagen in de boxplots voordat we de outliers hebben verwijderd.

Nu gaan we kijken of er na opvulling en opschoning van onze dataset logische clusters kunnen worden bepaald met unsupervised learning. De 2 modellen die we gebruiken zijn kmeans en gmm. We gebruiken deze 2 modellen omdat ze relatief eenvoudig, efficiënt en effectief zijn bij het vinden van clusters in onze continueu data. Kmeans kan goed voor bolvormige clusters modelleren en gmm kan goed verschillende vormen moddeleren doordat het flexibeler is.

In [20]:
# Code afkomstig van les CM10
# kMeans en GMM maken gebruik van afstandsmaten, daarom is standaardiseren belangrijk
df.reset_index(inplace = True)

# We passen scaling toe zodat de afstandmaten beter zijn verdeeld onder elkaar.
scaler = StandardScaler()
scaler.fit(df[['gross', 'budget']])
df_z = pd.DataFrame(scaler.transform(df[['gross', 'budget']]), columns=['gross_z', 'budget_z'])
df[['gross_z', 'budget_z']] = df_z

Kijken of de correcte kolommen zijn aangemaakt. gross_z en budget_z zijn inderdaad aangemaakt.

In [ ]:
df.head(5)

## Moddeleren

We gaan als eerst KMeans toepassen op onze dataset en bekijken welke clusters dit moddeleert. We kiezen hier handmatig 5 clusters. Dit doen we omdat onze hoofdvraag vraagt om 5 clusters: blockbuster films, flop films, cult films, mid range films en rest van de films.

In [ ]:
# Maken van een willekeurige clustering
model_kMeans = KMeans(n_clusters=5, random_state=0)
X_kMeans = df[['gross_z', 'budget_z']]

# Clusters 'voorspellen' en opslaan
prediction_kMeans = model_kMeans.fit_predict(X_kMeans)
df['cluster_number'] = model_kMeans.predict(X_kMeans)

# Plotten van 'omzet' en 'budget' en als kleur de clusters
plt.scatter(df['gross'], df['budget'], c=df['cluster_number'], cmap='plasma')
plt.gca().xaxis.set_major_formatter(FuncFormatter(fn.euro_formatter))
plt.gca().yaxis.set_major_formatter(FuncFormatter(fn.euro_formatter))
plt.title('Scatterplot: omzet vs budget')
plt.xlabel('omzet')
plt.ylabel('budget')
plt.show()

Als we naar de plot kijken zien we dat KMeans uitstekende logische clusters kan vormen. Alle kleuren staan gegroupeerd bij elkaar met logische parameters per groep. Helaas geeft dit geen clusters die overeenkomen met de 5 soorten films uit onze hoofdvraag. 

Om nog even bij KMeans te blijven gaan we hier de 'elbow method' toepassen. Dit doen we omdat dit het punt is waar het toevoegen van extra clusters nauwelijks meer leidt tot een significante verbetering in de clusteringkwaliteit. Als de 'knik' valt op 5 weten we dat dit het optimale punt is om te zoeken naar clusters.

In [ ]:
# Code afkomstig van les CM10

data = []

max_k = 20

for i in range(1, max_k):
    model_kMeans = KMeans(n_clusters=i, random_state=0)
    X_kMeans = df[['gross_z', 'budget_z']]

    # Clusters 'voorspellen' en opslaan
    model_kMeans.fit(X_kMeans)

    data.append([i, model_kMeans.score(X_kMeans)])

df_plot_kmeans = pd.DataFrame(data, columns=['k', 'Score'])

fig = plt.figure(figsize=(5,5), dpi=150)

ax = plt.axes()

ax.set(xlim=(0,max_k),
       xlabel='n',
       ylabel='Score',
       title='Kmeans: score vs k')

ax.xaxis.set_major_locator(plt.MaxNLocator(max_k))
ax.ticklabel_format(useOffset=False)
ax.plot(df_plot_kmeans['k'], df_plot_kmeans['Score'], '-o')

ax.legend(['k'])

Als we kijken naar de plot van de 'elbow method' zien we dat de 'knik' zit bij n=2. Dit punt geeft de optimale balans in modelcomplexiteit en clusteringkwaliteit. De knik valt dus niet op 5, wat betekend dat we eerder met n=5 hebben overclusterd. Dit betekent niet dat het geen logische clusters heeft gevonden, maar wel dat het overcomplexe clusters probeert te vinden zonder dat de kwaliteit van de clusters omhoog gaat. Hieronder plotten we de optimale balans met n=2.

In [ ]:
# Maken van een willekeurige clustering
model_kMeans = KMeans(n_clusters=2, random_state=0)
X_kMeans = df[['gross_z', 'budget_z']]

# Clusters 'voorspellen' en opslaan
prediction_kMeans = model_kMeans.fit_predict(X_kMeans)
df['cluster_number'] = model_kMeans.predict(X_kMeans)

# Plotten van 'omzet' en 'budget' en als kleur de clusters
plt.scatter(df['gross'], df['budget'], c=df['cluster_number'], cmap='plasma')
plt.gca().xaxis.set_major_formatter(FuncFormatter(fn.euro_formatter))
plt.gca().yaxis.set_major_formatter(FuncFormatter(fn.euro_formatter))
plt.title('Scatterplot: omzet vs budget')
plt.xlabel('omzet')
plt.ylabel('budget')
plt.show()

Ook dit geeft weer logische clusters zoals laag budget en omzet en hoog budget en omzet, maar geen clusters die slaan op onze hoofdvraag.

Nu gaan we hetzelfde doen maar dan met GMM.

In [ ]:
# Maken van een willekeurige clustering
model_gmm = gmm(n_components=5, random_state=0)
X_gmm = df[['gross_z', 'budget_z']]

# Clusters 'voorspellen' en opslaan
prediction_gmm = model_gmm.fit_predict(X_gmm)
df['cluster_number'] = model_gmm.predict(X_gmm)

# Plotten van 'omzet' en 'budget' en als kleur de clusters
plt.scatter(df['gross'], df['budget'], c=df['cluster_number'], cmap='plasma')
plt.gca().xaxis.set_major_formatter(FuncFormatter(fn.euro_formatter))
plt.gca().yaxis.set_major_formatter(FuncFormatter(fn.euro_formatter))
plt.title('Scatterplot: omzet vs budget')
plt.xlabel('omzet')
plt.ylabel('budget')
plt.show()

Ook hier zie je dat het clusters maakt die logisch zijn maar weer niet slaan op onze hoofdvraag. De vormen zijn wat flexibeler en rondvormig, wat een kenmerk is van gmm.

Dan gaan we kijken naar de elbow method voor gmm.

In [ ]:
# Code afkomstig van les CM10
data = []

max_n = 20

for i in range(1, max_k):
    model_gmm = gmm(n_components=i, random_state=0)
    X_gmm = df[['gross_z', 'budget_z']]

    prediction_gmm = model_gmm.fit(X_gmm)
    data.append([i, model_gmm.score(X_gmm)])

df_plot_gmm = pd.DataFrame(data, columns=['n', 'Score'])

fig = plt.figure(figsize=(5,5), dpi=150)

ax = plt.axes()

ax.set(xlim=(0,max_n),
       xlabel='n',
       ylabel='Score',
       title='GMM: score vs n')

ax.xaxis.set_major_locator(plt.MaxNLocator(20))
ax.ticklabel_format(useOffset=False)
ax.plot(df_plot_gmm['n'], df_plot_gmm['Score'], '-o')

ax.legend(['n'])

De knik bij de elbow method voor gmm zit op 4. Dit punt geeft de optimale balans in modelcomplexiteit en clusteringkwaliteit. De knik valt dus niet op 5, wat betekend dat we eerder met n=5 hebben overclusterd. Dit betekent niet dat het geen logische clusters heeft gevonden, maar wel dat het overcomplexe clusters probeert te vinden zonder dat de kwaliteit van de clusters omhoog gaat. Zeker bij deze want hier flatlined de plot tussen 4 en 5. Hieronder plotten we de optimale balans met n=4.

In [ ]:
# Maken van een willekeurige clustering
model_gmm = gmm(n_components=4, random_state=0)
X_gmm = df[['gross_z', 'budget_z']]

# Clusters 'voorspellen' en opslaan
prediction_gmm = model_gmm.fit_predict(X_gmm)
df['cluster_number'] = model_gmm.predict(X_gmm)

# Plotten van 'omzet' en 'budget' en als kleur de clusters
plt.scatter(df['gross'], df['budget'], c=df['cluster_number'], cmap='plasma')
plt.gca().xaxis.set_major_formatter(FuncFormatter(fn.euro_formatter))
plt.gca().yaxis.set_major_formatter(FuncFormatter(fn.euro_formatter))
plt.title('Scatterplot: omzet vs budget')
plt.xlabel('omzet')
plt.ylabel('budget')
plt.show()

Ook hier zie je dat het clusters maakt die logisch zijn maar weer niet slaan op onze hoofdvraag. De logische clusters hebben allemaal dezelfde combinatie van hoeveelheid in budget en omzet. De vormen zijn wat flexibeler en rondvormig, wat een kenmerk is van gmm.

Natuurlijk zouden we dit zelf wel kunnen maken op basis van een paar regels, namelijk:

Het opdelen van de films in 5 categorieën.

1. Blockbuster: hoog budget met hoge omzet
2. Flop: hoog budget met lage omzet
3. Cultfilm: laag budget met hoge omzet
4. Mid-Range Movie: Gemiddeld budget met gemiddelde omzet
5. Average: Alle andere gevallen

In [ ]:
# In volledige dataFrame, alle NaN waarde verwijderen van budget en gross.
df = dfr[['budget', 'gross']].dropna()

# Extreme waarde van budget en gross worden ook verwijderd.
df = df[(df['budget'] < 350000000) & (df['gross'] < 1000000000)] \

# Zie functies.py voor de functie classify_movie
df['Category'] = df.apply(fn.classify_movie, axis=1)

# Scatterplot Visualiseren met kleur
plt.figure(figsize=(12, 8))
colors = {'Blockbuster':'green', 'Flop':'red', 'Cultfilm':'blue', 'Average':'orange', 'Mid-Range Movie':'purple'}
scatter = plt.scatter(df['budget'], df['gross'], 
                      c=df['Category'].map(colors), alpha=0.5, s=60, edgecolor='k', marker='o')

# Titel en labels
plt.title('Filmclusters Gebaseerd op gudget en winst', fontsize=16)
plt.xlabel('Budget (in dollars)', fontsize=14)
plt.ylabel('Omzet (in dollars)', fontsize=14)

# Weergavegrenzen
plt.xlim(0, 300000000)  
plt.ylim(0, 800000000) 

# Grafiek lijn
plt.grid(True, linestyle='-', alpha=0.5)

plt.gca().xaxis.set_major_formatter(FuncFormatter(fn.euro_formatter))
plt.gca().yaxis.set_major_formatter(FuncFormatter(fn.euro_formatter))

plt.show()

**Kleur en kermerken van de clusters in de scatterplot**

- **Groen: blockbuster films**

Deze films zijn met een hoog budget (> 100 miljoen dollar) en met een hoge omzet (>300 miljoen dollar).

We zien groene cluster rechterboven in de grafiek. De succesvolle films.


- **Rood: flop films**

Deze films zijn met een hoog budget (> 100 miljoen dollar) en met een lage omzet (<50 miljoen dollar).

We zien rode cluster rechtsonder in de grafiek. Deze is een klein cluster.


- **Blauw: cultfilms**

Deze films zijn met een laag budget (<20 miljoen dollar) en met een relatief hoge omzet (>50 miljoen dollar).

We zien blauwe cluster linkerboven in de grafiek. Dit cluster laten zien de films met een laag budget toch tot een groot succes kan zijn.


- **Paars: mid range films**

Deze films zijn met een middelgroot budget (50-100 miljoen dollar) en met een middelgroot omzet (50-300 miljoen dollar).

We zien paarse cluster midden in de grafiek. Deze is een groot cluster, betekent er zijn heel veel prima films.


- **Orange: rest van de films**

Alle film die niet in bovenstaande categorieën.

We zien orange cluster verspreid over de grafiek. Er zijn heel veel films die gewoon middelmatige resultaten halen, dus niet te ondersheiden in budget of omzet.



## Conclusie onderzoeksvraag 3


Door middel van unsupervised learning kunnen er logische clusters gemaakt worden. Alle 2 de modellen presteren hier goed in. De kleuren zijn niet verspreid van elkaar wat dit aanduidt. Er is echter geen goed onderscheid te maken tussen blockbusters, flops en cultfilms als je kijkt naar de kleuren van de clusters. Zelfs met verschillende k waarden van beide modellen wordt het er niet beter op.

We kunnen wel door middel van rule based tabellen een logische cluster maken. Maar goed, dat is supervised en niet unsupervised learning.

Het is dus mogelijk om logische clusters te vinden met onze unsupervised technieken, maar deze kunnen geen onderscheid vinden tussen de 5 verschillende films.